# Application Task for Yellow Taxi Trop Records

Data Source: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

Objectives:
- Using Yellow Taxi Trips data, aggregate demand across NYC 
- measured by the count of Yellow Taxi trips commencing in a one-hour block. 
- For example, when given a time in the future (e.g. 7pm on 2025-05-01) this model should forecast the total
number of trips commencing between 7pm to 8pm on that date.

## Ch1: Data Exploration

Import and get familiar with the data. Make sure there are no errors with importing data.

In [457]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


df = pd.read_parquet("yellow_tripdata_2025-03.parquet") 
num_rows = df.shape[0]
num_cols = df.shape[1]
print(f'Yellow Taxi trip data has {num_cols} features and {num_rows} instances.')
df.head()

Yellow Taxi trip data has 20 features and 4145257 instances.


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-03-01 00:17:16,2025-03-01 00:25:52,1.0,0.90,1.0,N,140,236,1,7.9,3.50,0.5,2.60,0.0,1.0,15.50,2.5,0.0,0.00
1,1,2025-03-01 00:37:38,2025-03-01 00:43:51,1.0,0.60,1.0,N,140,262,1,6.5,3.50,0.5,2.30,0.0,1.0,13.80,2.5,0.0,0.00
2,2,2025-03-01 00:24:35,2025-03-01 00:39:49,1.0,1.94,1.0,N,161,68,1,14.9,1.00,0.5,5.16,0.0,1.0,25.81,2.5,0.0,0.75
3,2,2025-03-01 00:56:16,2025-03-01 01:01:35,2.0,0.95,1.0,N,231,13,1,7.2,1.00,0.5,2.59,0.0,1.0,15.54,2.5,0.0,0.75
4,1,2025-03-01 00:01:44,2025-03-01 00:10:00,1.0,1.50,1.0,N,163,236,1,8.6,4.25,0.5,2.85,0.0,1.0,17.20,2.5,0.0,0.75


### Check and clean data if appropriate
Consider features of interest and check for nans

In [458]:
print(f'Total data has {num_rows} instances.\n')
print('Number of nans:')
print(df.isna().sum())


Total data has 4145257 instances.

Number of nans:
VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          916663
trip_distance                 0
RatecodeID               916663
store_and_fwd_flag       916663
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge     916663
Airport_fee              916663
cbd_congestion_fee            0
dtype: int64


In [459]:
missing = round(max(df.isna().sum())/len(df) * 100,3)
print(f'Missing proportion: {missing}%')

Missing proportion: 22.114%


We can see that about 26% of the data has no data for specific columns, this could mean a certain type of Taxi does not support these features for data collection.

Since we only consider the features regarding demand of Yellow Taxi's, we can ignore the missing data columns.

**We focus on the 'tpep_pickup_datetime' as we assume the Pickup time aggrigates the demand.**

### Feature Exploration

We now engineer the **tpep_pickup_datetime** and extract the data by day and hour. 

In [460]:
# sort tpep_pickup_datetime just incase

feature = 'tpep_pickup_datetime'
df[feature] = pd.to_datetime(df[feature])
sort_df = df.sort_values(by=feature)

sort_df.tail()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
4143632,2,2025-03-31 23:59:58,2025-04-01 00:05:55,NaN,1.17,NaN,None,230,50,0,11.52,0.00,0.5,0.0,0.0,1.0,16.27,NaN,NaN,0.75
3226432,1,2025-03-31 23:59:59,2025-04-01 00:06:07,1.0,0.90,1.0,N,230,246,1,6.50,3.25,0.5,2.0,0.0,1.0,13.25,2.5,0.00,0.75
3226221,2,2025-03-31 23:59:59,2025-04-01 00:05:26,1.0,1.24,1.0,N,230,143,1,7.90,1.00,0.5,2.0,0.0,1.0,15.65,2.5,0.00,0.75
3226205,2,2025-04-01 00:00:07,2025-04-01 00:24:28,1.0,10.40,1.0,N,132,82,2,42.90,1.00,0.5,0.0,0.0,1.0,47.15,0.0,1.75,0.00
3227804,2,2025-04-01 00:00:17,2025-04-01 00:18:42,5.0,3.68,1.0,N,43,166,2,20.50,1.00,0.5,0.0,0.0,1.0,26.25,2.5,0.00,0.75


Although incredibly minor, we have to remove all taxi start times before 2025-05-01 00:00:00 and after 2025-05-31 23:59:59

In [461]:
start_date = pd.Timestamp("2025-03-01 00:00:00")
end_date = pd.Timestamp("2025-03-31 23:59:59")

sdf = sort_df
filterd_df = sdf[(sdf[feature] >= start_date) & (sdf[feature] <= end_date)]
removed_inst = len(sdf) - len(filterd_df)
print(f'Removed {removed_inst} instances that were outside {start_date} and {end_date}.')

Removed 33 instances that were outside 2025-03-01 00:00:00 and 2025-03-31 23:59:59.


In [462]:
filterd_df.loc[:, 'date_hours'] = filterd_df[feature].dt.strftime('%d %H').copy()
datehour_counts = filterd_df.groupby('date_hours').size().reset_index(name='count')
datehour_counts.head()

### Visualisation

Now we have the data in our desired format

In [464]:
import plotly.graph_objects as go

# Plotly line chart
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=datehour_counts['date_hours'],
    y=datehour_counts['count'],
    mode='lines+markers',
    name='Hourly Counts'
))

fig.update_layout(
    title='Hourly Activity Counts (March 2025)',
    xaxis_title='Date Hour (DD HH)',
    yaxis_title='Count',
    xaxis_tickangle=-45
)

fig.show()


We can see a cyclical pattern with this data as times from 5pm - 12am have a lot busier than early in the morning. 

Looking closely at the daily minimums, you can see a weekly shift in the data, where more taxi's are called on Saturday and Sunday morning.

This motivates us to use 2 fourier trends; A daily frequency and a weekly frequency.

## Ch2. Strategy and Objectives
Predict the 2025 March Taxi trips aggregated hourly.

There are 3 main trends to consider when considering a prediction:
- Yearly; Predict the yearly change of taxi trips from year to year
- Week; Weekends will have higher taxi trips
- Daily; less taxi trips during 2-5am

### Objective:
Predict the Taxi Trips for all from 2025 March

## Ch3. Feature Engineering and Extraction
Extract the last 4 years of NY Taxi data from Feb and March.

In [465]:
import os

# Extract data from folder
def data_df(file_name = ''):
    if file_name[-10:-8] == '02':
        start = file_name[-13:-8]
        if file_name[-13:-11] == '24':
            end = file_name[-13:-8] + '-29'
        else:
            end = file_name[-13:-8] + '-28'
    elif file_name[-10:-8] == '03':
        start = file_name[-13:-8]
        end = file_name[-13:-8] + '-31'
    df = pd.read_parquet(file_name) 
    feature = 'tpep_pickup_datetime'
    df[feature] = pd.to_datetime(df[feature])
    sdf = df.sort_values(by=feature)
    start_date = pd.Timestamp(f"20{start}-01 00:00:00")
    end_date = pd.Timestamp(f"20{end} 23:59:59")
    filterd_df = sdf[(sdf[feature] >= start_date) & (sdf[feature] <= end_date)].copy()
    filterd_df.loc[:, 'date_hours'] = filterd_df[feature].dt.strftime('%Y %m %d %H').copy()
    datehour_counts = filterd_df.groupby('date_hours').size().reset_index(name='count')
    return datehour_counts

# Automate extraction
data_dict = {}
for file in os.listdir():
    if os.path.isfile(file) and file[-8:] == '.parquet':
        data_dict[file[-13:-8]] = data_df(file_name = file)


In [540]:
year_dict = defaultdict(list)
for key, df in data_dict.items():
    year = key[:2]
    year_dict[year].append(df)
yearly_data = {}
for year, df_list in year_dict.items():
    yearly_data[year] = pd.concat(df_list, ignore_index=True)


We now have data from 2022-2025 Feb and March, its actually more important to get the day of the week correct. 

If we train our data on date specific, all previous years will be off by a day. 

i.e March 1st 2025 is a Saturday but March 1st 2024 is a Friday. The DATE doesnt give predictions, its the **day of the week**.

Thus we have to scale the training dates to begin on Saturday (01/03/2025) and finish on Monday (31/03/2025)

In [541]:
def align_by_weekday(df, target_year=2025, min_rows=1319):
    df = df.copy()
    df['dt'] = pd.to_datetime(df['date_hours'], format="%Y %m %d %H")

    # Get current year's 1 March weekday
    original_year = df['dt'].dt.year.min()
    original_march_first = pd.Timestamp(f"{original_year}-03-01")
    original_weekday = original_march_first.weekday()  # Monday=0, Sunday=6

    # Target weekday = weekday of 1 March 2025
    target_march_first = pd.Timestamp(f"{target_year}-03-01")
    target_weekday = target_march_first.weekday()

    # Compute delta using true signed difference (allows forward/backward shift)
    delta_days = target_weekday - original_weekday

    df['dt_aligned'] = df['dt'] + pd.Timedelta(days=delta_days)
    df['date_hours'] = df['dt_aligned'].dt.strftime('%Y %m %d %H')

    end_date = pd.Timestamp(f"{original_year}-03-31 23:59:59")
    mask = df['dt_aligned'] <= end_date
    df = df.loc[mask].copy()

    # Trim top to match min_rows (if provided)
    if min_rows is not None and len(df) > min_rows:
        df = df.iloc[-min_rows:].reset_index(drop=True)

    return df.drop(columns=['dt', 'dt_aligned'])
yearly_data = {year: align_by_weekday(df) for year, df in yearly_data.items()}

In [544]:
df_2025 = yearly_data['25'].copy()
df_2025['dt'] = pd.to_datetime(df_2025['date_hours'], format="%Y %m %d %H")
df_2025['time'] = df_2025['dt'].dt.strftime("%m-%d %H")

final_df = pd.DataFrame({
    'time': df_2025['time'].values,
    '2025': df_2025['count'].values
})
for yr in ['22', '23', '24']:
    df = yearly_data[yr].copy().reset_index(drop=True)
    final_df[f'20{yr}'] = df['count'].values
combined_df = final_df[['time', '2022', '2023', '2024', '2025']]
combined_df.head()


,time,2022,2023,2024,2025
0,02-05 00,1155,1707,1010,1275
1,02-05 01,555,853,518,530
2,02-05 02,318,479,290,282
3,02-05 03,225,310,202,175
4,02-05 04,188,310,214,303


In [573]:
fig = go.Figure()

combined_df = combined_df.copy()
combined_df['dt'] = pd.to_datetime(combined_df['time'], format="%m-%d %H")

# Sort by datetime and generate x-axis label
combined_df = combined_df.sort_values('dt').reset_index(drop=True)
combined_df['x_label'] = combined_df['dt'].dt.strftime("%m-%d %H")

# Plot each year column
for col in sorted([c for c in combined_df.columns if c not in ['time', 'dt', 'x_label']], reverse=True):
    fig.add_trace(
        go.Scatter(
            x=combined_df['x_label'],
            y=combined_df[col],
            mode='lines',
            name=f"{col}"
        )
    )

fig.update_layout(
    title="Taxi Trips correctly aligned by Day (MM-DD HH)",
    xaxis_title="Date (MM-DD HH)",
    yaxis_title="Trip Count",
    template="plotly_white",
    height=600,
    width=1500,
    xaxis=dict(tickangle=45)
)

fig.show()


In [547]:
# split data from Feb and March
combined_df['time'] = combined_df['time'].astype(str)
feb_mask = combined_df['time'].str.startswith('02')
mar_mask = combined_df['time'].str.startswith('03')
feb_df = combined_df[feb_mask].reset_index(drop=True)
march_df = combined_df[mar_mask].reset_index(drop=True)
feb_df.head()


,time,2022,2023,2024,2025,dt,x_label
0,02-05 00,1155,1707,1010,1275,1900-02-05 00:00:00,02-05 00
1,02-05 01,555,853,518,530,1900-02-05 01:00:00,02-05 01
2,02-05 02,318,479,290,282,1900-02-05 02:00:00,02-05 02
3,02-05 03,225,310,202,175,1900-02-05 03:00:00,02-05 03
4,02-05 04,188,310,214,303,1900-02-05 04:00:00,02-05 04


## Ch4. Model fitting, Fourier Transformations
We will calibrate a Fourier Transformation, enhancing it with our knowledge that the model follows daily and weekly cycle trends. Use RMSE as a loss function.

Separate the data with Feb and March, we use Feb data to calibrate our double Fourier model and train our calibrated model on the 2022-2024 March data.

Use Feb data to predict calibrate the double fourier series to minimise the RMSE. Using the calibration for 

In [567]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

def run_double_fourier_model(train_series, test_series, N_day=20, N_week=6):
    """
    Fit a linear regression model using double Fourier harmonics.

    Parameters:
    - train_series (pd.Series): Series of counts for training (indexed or indexed with time).
    - test_series (pd.Series): Series of counts for testing (aligned to same index as training).
    - N_day (int): Number of daily harmonics.
    - N_week (int): Number of weekly harmonics.

    Returns:
    - rmse_pct (float): RMSE on test set in percentage terms.
    - test_df (pd.DataFrame): Test data with prediction and error columns added.
    """

    # Create training dataframe
    df_train = pd.DataFrame({
        'count': train_series.values,
        't': np.arange(len(train_series))
    })

    # Test dataframe, assume same time index
    df_test = pd.DataFrame({
        'count': test_series.values,
        't': np.arange(len(test_series))  # could be continuous or restarted
    })

    def add_double_fourier(df_inner, N_day, N_week):
        for k in range(1, N_day + 1):
            df_inner[f'sin_day_{k}'] = np.sin(2 * np.pi * k * df_inner['t'] / 24)
            df_inner[f'cos_day_{k}'] = np.cos(2 * np.pi * k * df_inner['t'] / 24)
        for k in range(1, N_week + 1):
            df_inner[f'sin_week_{k}'] = np.sin(2 * np.pi * k * df_inner['t'] / 168)
            df_inner[f'cos_week_{k}'] = np.cos(2 * np.pi * k * df_inner['t'] / 168)
        return df_inner

    df_train = add_double_fourier(df_train, N_day, N_week)
    df_test = add_double_fourier(df_test, N_day, N_week)

    # Select harmonics
    fourier_cols = (
        [f'sin_day_{k}' for k in range(1, N_day + 1)] +
        [f'cos_day_{k}' for k in range(1, N_day + 1)] +
        [f'sin_week_{k}' for k in range(1, N_week + 1)] +
        [f'cos_week_{k}' for k in range(1, N_week + 1)]
    )

    # Train and predict
    model = LinearRegression().fit(df_train[fourier_cols], df_train['count'])
    df_test['fourier_pred'] = model.predict(df_test[fourier_cols])
    df_test['error'] = (df_test['fourier_pred'] - df_test['count'])

    rmse_pct = np.sqrt(np.mean(df_test['error'] ** 2))

    return rmse_pct, df_test

rmse, data = run_double_fourier_model(feb_df['2022'], feb_df['2025'], N_day=20, N_week=6)
print(rmse)

1924.7788529111956


In [568]:
def calibrate_fourier_model(train_series, test_series, day_range=range(1, 21), week_range=range(1, 11), verbose=False):
    """
    Grid search to find best (N_day, N_week) based on minimum test RMSE.

    Parameters:
    - train_series (pd.Series): Series of training counts (e.g., from 2022-2024)
    - test_series (pd.Series): Series of test counts (e.g., 2025)
    - day_range (range): Range of N_day values to test
    - week_range (range): Range of N_week values to test
    - verbose (bool): Print intermediate RMSE values

    Returns:
    - best_params (tuple): (best_N_day, best_N_week)
    - best_rmse (float): Best RMSE found
    - results (list of dict): All RMSEs with corresponding params
    """
    results = []
    best_rmse = float('inf')
    best_params = (None, None)

    for N_day in day_range:
        for N_week in week_range:
            try:
                rmse, _ = run_double_fourier_model(train_series, test_series, N_day=N_day, N_week=N_week)
                results.append({'N_day': N_day, 'N_week': N_week, 'rmse_pct': rmse})
                if verbose:
                    print(f"N_day={N_day}, N_week={N_week} -> RMSE: {rmse:.2f}")
                if rmse < best_rmse:
                    best_rmse = rmse
                    best_params = (N_day, N_week)
            except Exception as e:
                if verbose:
                    print(f"Failed at N_day={N_day}, N_week={N_week}: {e}")

    print(f"Best combination: N_day={best_params[0]}, N_week={best_params[1]} with RMSE={best_rmse:.2f}")
    return best_params, best_rmse, results

    

In [569]:
# Store results
all_test_results = {}
calibration_summary = {}

# Loop through each year
for year in ['2022', '2023', '2024']:
    print(f"\nCalibrating with {year} against 2025:")
    best_params, best_rmse, all_results = calibrate_fourier_model(
        feb_df[year],
        feb_df['2025'],
        day_range=range(1, 31),
        week_range=range(1, 31),
        verbose=False
    )

    calibration_summary[year] = {
        'best_N_day': best_params[0],
        'best_N_week': best_params[1],
        'rmse': best_rmse
    }

    rmse, test_df = run_double_fourier_model(
        feb_df[year],
        feb_df['2025'],
        N_day=best_params[0],
        N_week=best_params[1]
    )
    all_test_results[year] = test_df


fig = go.Figure()

test_actual = feb_df['2025'].reset_index(drop=True)
fig.add_trace(go.Scatter(
    y=test_actual,
    x=list(range(len(test_actual))),
    mode='lines',
    name='2025 Actual',
    line=dict(color='black', width=3, dash='dot')
))

for year, df in all_test_results.items():
    fig.add_trace(go.Scatter(
        y=df['fourier_pred'],
        x=list(range(len(df))),
        mode='lines',
        name=f"Predicted from {year} (RMSE: {calibration_summary[year]['rmse']:.2f})"
    ))

fig.update_layout(
    title="Fourier Calibration: Predicted 2025 from Different Years",
    xaxis_title="Hour Index",
    yaxis_title="Trip Count",
    template="plotly_white",
    height=500,
    width=1000,
    legend_title="Model Source"
)

fig.show()


Calibrating with 2022 against 2025:
Best combination: N_day=8, N_week=15 with RMSE=1864.94

Calibrating with 2023 against 2025:
Best combination: N_day=19, N_week=15 with RMSE=1964.93

Calibrating with 2024 against 2025:
Best combination: N_day=10, N_week=1 with RMSE=2043.85


### Findings for Double Fourier Calibration
We can see the best calibration was 2022 with N_day = 8 and N_week = 15.

However you can see there is a distinct increase on the 2025 data.

Without over complicating the model, let's multiply our training data by a scalar multiplyer. i.e 2025_data = alpha * 2023_data

In [570]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

def plot_scalar_adjustment(train_series, test_series, N_day, N_week, alpha, title_suffix=""):
    """
    Plots the actual vs predicted counts (scaled and unscaled),
    and plots both residuals (scaled and unscaled) in a subplot.
    """
    # Run model and prepare data
    _, test_df = run_double_fourier_model(train_series, test_series, N_day=N_day, N_week=N_week)

    actual = test_series.reset_index(drop=True)
    pred = test_df['fourier_pred']
    scaled_pred = alpha * pred

    # Residuals
    residual_unscaled = pred - actual
    residual_scaled = scaled_pred - actual

    # RMSE
    rmse_original = np.sqrt(np.mean(residual_unscaled ** 2))
    rmse_scaled = np.sqrt(np.mean(residual_scaled ** 2))

    # Create subplots
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        row_heights=[0.7, 0.3],
        vertical_spacing=0.1,
        subplot_titles=(
            f"Fourier Prediction ({title_suffix}) → RMSE from {rmse_original:.2f} to {rmse_scaled:.2f}",
            "Residuals: (Prediction − Actual)"
        )
    )

    # Top Plot: Predictions vs Actual
    fig.add_trace(go.Scatter(
        x=list(range(len(actual))),
        y=actual,
        mode='lines',
        name='2025 Actual',
        line=dict(color='black', width=3, dash='dot')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=list(range(len(pred))),
        y=pred,
        mode='lines',
        name='Prediction (no scaling)',
        line=dict(color='red')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=list(range(len(pred))),
        y=scaled_pred,
        mode='lines',
        name=f'Prediction (scaled, α={alpha:.3f})',
        line=dict(color='orange')
    ), row=1, col=1)

    # Bottom Plot: Residuals
    fig.add_trace(go.Scatter(
        x=list(range(len(pred))),
        y=residual_unscaled,
        mode='lines',
        name='Residual (unscaled)',
        line=dict(color='red', dash='dot')
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=list(range(len(pred))),
        y=residual_scaled,
        mode='lines',
        name='Residual (scaled)',
        line=dict(color='orange', dash='dot')
    ), row=2, col=1)

    # Layout
    fig.update_layout(
        height=800,
        width=1000,
        template="plotly_white",
        xaxis2_title="Hour Index (Feb-Mar)",
        yaxis1_title="Trip Count",
        yaxis2_title="Residuals",
        showlegend=True
    )

    fig.show()


In [571]:
calibration_summary = {}

for year in ['2022', '2023', '2024']:
    best_params, best_rmse, all_results = calibrate_fourier_model_with_scalar(
        feb_df[year], feb_df['2025'],
        day_range=range(1, 31),
        week_range=range(1, 31),
        verbose=False,
        year=year
    )

    calibration_summary[year] = {
        'params': best_params,
        'rmse': best_rmse,
        'results': all_results
    }
    
    # Plot comparisons
    plot_scalar_adjustment(
        train_series=feb_df[year],
        test_series=feb_df['2025'],
        N_day=best_params['N_day'],
        N_week=best_params['N_week'],
        alpha=best_params['alpha'],
        title_suffix=f"trained on {year}"
    )


Year 2022: Best Params → N_day=8, N_week=15, Alpha=1.193, RMSE=1592.35


Year 2023: Best Params → N_day=18, N_week=1, Alpha=1.212, RMSE=1693.11


Year 2024: Best Params → N_day=10, N_week=1, Alpha=1.240, RMSE=1685.76


### Findings Double Fourier Transform from Feb Data
From the results, the best result was when 2022 year with the calibration of N_day=11, N_week=15, Alpha=1.181. 

Meaning we will approximate the March 2025 data with the March 2022 data with the calibrations found above.

## Ch5. Predictions and results

In [572]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

def run_final_march_prediction_datetime_x(march_df, train_yr='2022', test_yr='2025', N_day=11, N_week=15, alpha=1.181):
    """
    Train on March 2022, predict March 2025 using calibrated Fourier parameters and scalar alpha.
    X-axis uses datetime labels for March 2025. Also plots residuals.

    Returns:
    - test_df (pd.DataFrame): Test dataframe with prediction and residuals
    - rmse_scaled (float): RMSE after applying alpha
    """

    # Prepare training and test series
    train_series = march_df[train_yr].copy().reset_index(drop=True)
    test_series = march_df[test_yr].copy().reset_index(drop=True)

    # Generate datetime index
    start_date = pd.Timestamp("2025-03-01 00:00:00")
    dt_index = pd.date_range(start=start_date, periods=len(test_series), freq="H")

    def add_double_fourier(df, N_day, N_week):
        df = df.copy()
        df['t'] = np.arange(len(df))
        for k in range(1, N_day + 1):
            df[f'sin_day_{k}'] = np.sin(2 * np.pi * k * df['t'] / 24)
            df[f'cos_day_{k}'] = np.cos(2 * np.pi * k * df['t'] / 24)
        for k in range(1, N_week + 1):
            df[f'sin_week_{k}'] = np.sin(2 * np.pi * k * df['t'] / 168)
            df[f'cos_week_{k}'] = np.cos(2 * np.pi * k * df['t'] / 168)
        return df

    # Build dataframes
    df_train = train_series.to_frame(name='count')
    df_test = test_series.to_frame(name='count')
    df_test['datetime'] = dt_index

    df_train = add_double_fourier(df_train, N_day, N_week)
    df_test = add_double_fourier(df_test, N_day, N_week)

    fourier_cols = (
        [f'sin_day_{k}' for k in range(1, N_day + 1)] +
        [f'cos_day_{k}' for k in range(1, N_day + 1)] +
        [f'sin_week_{k}' for k in range(1, N_week + 1)] +
        [f'cos_week_{k}' for k in range(1, N_week + 1)]
    )

    # Train and predict
    model = LinearRegression()
    model.fit(df_train[fourier_cols], df_train['count'])
    df_test['fourier_pred'] = model.predict(df_test[fourier_cols])
    df_test['scaled_pred'] = alpha * df_test['fourier_pred']
    df_test['residual'] = df_test['scaled_pred'] - df_test['count']

    # Compute RMSE
    rmse_scaled = np.sqrt(np.mean(df_test['residual'] ** 2))

    # Main prediction plot
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(
        x=df_test['datetime'], y=df_test['count'],
        mode='lines', name=f'{test_yr} Actual',
        line=dict(color='black', width=3, dash='dot')
    ))
    fig1.add_trace(go.Scatter(
        x=df_test['datetime'], y=df_test['scaled_pred'],
        mode='lines', name='Prediction',
        line=dict(color='green')
    ))
    fig1.update_layout(
        title=f"Prediction: Mar {train_yr} → Mar {test_yr} | RMSE: {rmse_scaled:.2f}",
        xaxis_title="Datetime",
        yaxis_title="Trip Count",
        height=500,
        template="plotly_white"
    )

    # Residual plot
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=df_test['datetime'], y=df_test['residual'],
        mode='lines', name='Residuals',
        line=dict(color='red')
    ))
    fig2.update_layout(
        title="Residuals (Predicted - Actual)",
        xaxis_title="Datetime",
        yaxis_title="Residual (Count)",
        height=300,
        template="plotly_white"
    )

    fig1.show()
    fig2.show()

    return df_test, rmse_scaled


In [558]:
final_df, rmse = run_final_march_prediction_datetime_x(march_df, train_yr = '2022', test_yr = '2025', N_day=8, N_week=15, alpha=1.193)

## Comments on the Results
We discovered the model that best predicted the 2025 March data is the 2022 March data.

We built a custom Double Fourier model to predict the March data and used ML to calibrate the hyper-parameters.

Weaknesses in the model are Sundays where our prediction over estimates the Taxi trips and some higher frequency features were no captured in the model.

Feel free to adjust the hyper-parameters to find a different solution that would decrease the RMSE.